In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('dataset_sample_500.csv')  

# Podstawowe informacje o danych
print("Rozmiar danych:", data.shape)
print("\nTypy kolumn:\n", data.dtypes)
print("\nBrakujące wartości:\n", data.isnull().sum())

# Rozkłady cech numerycznych
num_cols = data.select_dtypes(include=['float64', 'int64']).columns
data[num_cols].hist(figsize=(15, 10), bins=20)
plt.suptitle("Rozkłady cech numerycznych")
plt.show()

# Rozkłady cech kategorycznych
cat_cols = data.select_dtypes(include=['object']).columns
for col in cat_cols:
    plt.figure(figsize=(10, 5))
    sns.countplot(data[col])
    plt.title(f"Rozkład zmiennej {col}")
    plt.xticks(rotation=45)
    plt.show()

# Wskazanie targetu
target = 'xxx'  
print(f"Zmienna docelowa: {target}")
print(data[target].value_counts())

# Wstępne czyszczenie danych
# 1. Uzupełnianie brakujących wartości
for col in data.columns:
    if data[col].isnull().sum() > 0:
        if data[col].dtype in ['float64', 'int64']:
            data[col].fillna(data[col].median(), inplace=True)
        else:
            data[col].fillna(data[col].mode()[0], inplace=True)

# 2. Enkodowanie zmiennych kategorycznych
data = pd.get_dummies(data, columns=cat_cols, drop_first=True)

# 3. Usunięcie wartości odstających 
from scipy.stats import zscore
z_scores = np.abs(zscore(data[num_cols]))
data = data[(z_scores < 3).all(axis=1)]

# Podsumowanie czyszczenia
print("Rozmiar danych po czyszczeniu:", data.shape)

# Wybór metryki ewaluacji
# Zakładamy, że problem to klasyfikacja binarna
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score

X = data.drop(columns=[target])
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

roc_auc = roc_auc_score(y_test, y_pred_proba)
f1 = f1_score(y_test, y_pred)

print(f"ROC-AUC: {roc_auc:.2f}")
print(f"F1-Score: {f1:.2f}")